<a href="https://colab.research.google.com/github/Ali-Ahmed-xixa/301-redirect-url-CSV-mapper-paython/blob/main/shopware%20to%20jtl%20migration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
from datetime import datetime

def map_shopware_to_jtl(shopware_file, output_file):
    # JTL headers in correct order
    jtl_headers = [
        'kKundengruppe', 'kSprache', 'cKundenNr', 'cPasswort',
        'cAnrede', 'cTitel', 'cVorname', 'cNachname', 'cFirma', 'cZusatz',
        'cStrasse', 'cHausnummer', 'cAdressZusatz', 'cPLZ', 'cOrt',
        'cBundesland', 'cLand', 'cTel', 'cMobil', 'cFax', 'cMail', 'cUSTID',
        'cWWW', 'cSperre', 'fGuthaben', 'cNewsletter', 'dGeburtstag',
        'fRabatt', 'cHerkunft', 'dErstellt', 'dVeraendert', 'cAktiv',
        'cAbgeholt', 'nRegistriert', 'nLoginversuche', 'dLastLogin',
        'dSessionInvalidate', 'b2FAauth', 'c2FAauthSecret'
    ]

    # Explicit field mapping with transformations
    field_mapping = {
        'customernumber': ('cKundenNr', str),
        'email': ('cMail', str),
        'password': ('cPasswort', str),
        'active': ('cAktiv', lambda x: 'Y' if str(x) == '1' else 'N'),
        'billing_company': ('cFirma', str),
        'billing_department': ('cZusatz', str),
        'billing_salutation': ('cAnrede', lambda x: 'Herr' if str(x).lower() == 'mr' else 'Frau' if str(x).lower() == 'ms' else str(x)),
        'billing_firstname': ('cVorname', str),
        'billing_lastname': ('cNachname', str),
        'billing_street': ('cStrasse', str),
        'billing_zipcode': ('cPLZ', str),
        'billing_city': ('cOrt', str),
        'phone': ('cTel', str),
        'fax': ('cFax', str),
        'ustid': ('cUSTID', str),
        'newsletter': ('cNewsletter', lambda x: '1' if str(x) == '1' else '0'),
        'customergroup': ('kKundengruppe', str),
        'language': ('kSprache', str)
    }

    with open(shopware_file, mode='r', encoding='utf-8') as infile, \
         open(output_file, mode='w', encoding='utf-8', newline='') as outfile:

        # Read first line to detect delimiter
        first_line = infile.readline()
        delimiter = '\t' if '\t' in first_line else ','
        infile.seek(0)

        # Read Shopware CSV
        reader = csv.DictReader(infile, delimiter=delimiter)
        print(f"Detected Shopware headers: {reader.fieldnames}")

        # Verify required fields exist
        missing_fields = [f for f in field_mapping if f not in reader.fieldnames]
        if missing_fields:
            raise ValueError(f"Missing required fields in Shopware CSV: {missing_fields}")

        # Write JTL CSV
        writer = csv.DictWriter(outfile, fieldnames=jtl_headers, delimiter=';')
        writer.writeheader()

        for idx, row in enumerate(reader, start=1):
            jtl_row = {h: '' for h in jtl_headers}

            # Set automatic fields
            jtl_row['dErstellt'] = datetime.now().strftime('%Y-%m-%d')
            jtl_row['dVeraendert'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            jtl_row['cSperre'] = 'N'
            jtl_row['fGuthaben'] = '0.00'
            jtl_row['fRabatt'] = '0.00'
            jtl_row['nRegistriert'] = '0'

            # Map all fields
            for shopware_field, (jtl_field, transform) in field_mapping.items():
                if shopware_field in row:
                    try:
                        value = row[shopware_field].strip() if row[shopware_field] else ''
                        if value:
                            jtl_row[jtl_field] = transform(value)
                    except Exception as e:
                        print(f"Error transforming {shopware_field}: {e}")
                        jtl_row[jtl_field] = ''

            # Handle street/house number
            if jtl_row['cStrasse']:
                street = jtl_row['cStrasse']
                parts = street.rsplit(' ', 1)
                if len(parts) > 1 and parts[1][0].isdigit():
                    jtl_row['cStrasse'] = parts[0]
                    jtl_row['cHausnummer'] = parts[1]

            writer.writerow(jtl_row)

            # Print first row for verification
            if idx == 1:
                print("\nFirst converted row:")
                for k, v in jtl_row.items():
                    if v: print(f"{k}: {v}")

    print(f"\nSuccessfully converted {idx} records to {output_file}")

if __name__ == "__main__":
    map_shopware_to_jtl(
        shopware_file='s_user_new.csv',
        output_file='jtl_export_7_01_2025.csv'
    )

Detected Shopware headers: ['id', 'password', 'encoder', 'email', 'active', 'accountmode', 'confirmationkey', 'paymentID', 'doubleOptinRegister', 'doubleOptinEmailSentDate', 'doubleOptinConfirmDate', 'firstlogin', 'lastlogin', 'sessionID', 'newsletter', 'validation', 'affiliate', 'customergroup', 'paymentpreset', 'language', 'subshopID', 'referer', 'pricegroupID', 'internalcomment', 'failedlogins', 'lockeduntil', 'default_billing_address_id', 'default_shipping_address_id', 'title', 'salutation', 'firstname', 'lastname', 'birthday', 'customernumber', 'login_token', 'changed', 'password_change_date', 'register_opt_in_id']


ValueError: Missing required fields in Shopware CSV: ['billing_company', 'billing_department', 'billing_salutation', 'billing_firstname', 'billing_lastname', 'billing_street', 'billing_zipcode', 'billing_city', 'phone', 'fax', 'ustid']

In [ ]:
import csv
from datetime import datetime

def convert_shopware_to_jtl(shopware_csv_path, jtl_csv_path):
    # Define JTL Shop 5 header
    jtl_header = [
        'kBestellung', 'kWarenkorb', 'kKunde', 'kLieferadresse', 'kRechnungsadresse',
        'kZahlungsart', 'kVersandart', 'kSprache', 'kWaehrung', 'fGuthaben', 'fGesamtsumme',
        'cSession', 'cVersandartName', 'cZahlungsartName', 'cBestellNr', 'cVersandInfo',
        'nLongestMinDelivery', 'nLongestMaxDelivery', 'dVersandDatum', 'dBezahltDatum',
        'dBewertungErinnerung', 'cTracking', 'cKommentar', 'cLogistiker', 'cTrackingURL',
        'cIP', 'cAbgeholt', 'cStatus', 'dErstellt', 'fWaehrungsFaktor', 'cPUIZahlungsdaten'
    ]

    def get_safe_value(row, key, default=''):
        """Safely get value from row with case-insensitive key matching"""
        for k, v in row.items():
            if k.lower() == key.lower():
                return v
        return default

    def transform_order(shopware_order):
        # Status mapping (Shopware → JTL)
        status_map = {
            '0': '0',   # Open
            '1': '1',   # In process
            '2': '2',   # Completed
            '3': '3',   # Partially completed
            '4': '4',   # Cancelled
            '5': '5',   # Ready for shipping
            '6': '6',   # Partially shipped
            '7': '2',   # Completed (from your example)
            '12': '2'   # Completed (Shopware cleared status)
        }

        # Payment method names
        payment_names = {
            '3': 'Vorkasse',
            '5': 'Nachnahme',
            '7': 'Überweisung'
        }

        # Shipping method names
        shipping_names = {
            '3': 'DHL',
            '9': 'Standard'
        }


        return {
            'kBestellung': get_safe_value(shopware_order, 'id'),
            'kWarenkorb': '3',
            'kKunde': get_safe_value(shopware_order, 'userID'),
            'kLieferadresse': '0',
            'kRechnungsadresse': '3',
            'kZahlungsart': get_safe_value(shopware_order, 'paymentID'),
            'kVersandart': get_safe_value(shopware_order, 'dispatchID'),
            'kSprache': '1' if get_safe_value(shopware_order, 'language') == '1' else '7',
            'kWaehrung': '1' if get_safe_value(shopware_order, 'currency') == 'EUR' else '2',
            'fGuthaben': '0',
            'fGesamtsumme': get_safe_value(shopware_order, 'invoice_amount'),
            'cSession': '9sk7f2fprla53cj37nn6okhhac',
            'cVersandartName': shipping_names.get(get_safe_value(shopware_order, 'dispatchID'), 'Standard'),
            'cZahlungsartName': payment_names.get(get_safe_value(shopware_order, 'paymentID'), 'Überweisung'),
            'cBestellNr': get_safe_value(shopware_order, 'ordernumber'),
            'cVersandInfo': get_safe_value(shopware_order, 'trackingcode'),
            'nLongestMinDelivery': '2',
            'nLongestMaxDelivery': '3',
            'dVersandDatum': '',
            'dBezahltDatum': get_safe_value(shopware_order, 'cleareddate').split()[0] if get_safe_value(shopware_order, 'cleareddate') else '',
            'dBewertungErinnerung': 'NULL',
            'cTracking': get_safe_value(shopware_order, 'trackingcode'),
            'cKommentar': get_safe_value(shopware_order, 'comment', 'NULL'),
            'cLogistiker': '',
            'cTrackingURL': '',
            'cIP': get_safe_value(shopware_order, 'remote_addr'),
            'cAbgeholt': 'Y',
            'cStatus': status_map.get(get_safe_value(shopware_order, 'status', '0'), '0'),
            'dErstellt': get_safe_value(shopware_order, 'ordertime'),
            'fWaehrungsFaktor': get_safe_value(shopware_order, 'currencyFactor', '1'),
            'cPUIZahlungsdaten': get_safe_value(shopware_order, 'transactionID')
        }

    # First detect the delimiter
    with open(shopware_csv_path, mode='r', encoding='utf-8') as f:
        dialect = csv.Sniffer().sniff(f.read(1024))
        f.seek(0)

        # Read input and write output
        with open(jtl_csv_path, mode='w', encoding='utf-8', newline='') as jtl_file:
            shopware_reader = csv.DictReader(f, delimiter=dialect.delimiter)
            jtl_writer = csv.DictWriter(jtl_file, fieldnames=jtl_header, delimiter='\t')

            jtl_writer.writeheader()

            for shopware_order in shopware_reader:
                jtl_order = transform_order(shopware_order)
                jtl_writer.writerow(jtl_order)

if __name__ == '__main__':
    convert_shopware_to_jtl('s_order_new.csv', 'jtl_orders_d.csv')

In [ ]:
import pandas as pd
from typing import Dict

def update_jtl_ids_only(shopware_file: str, jtl_file: str, output_file: str):
    """
    Updates ONLY the kKunde IDs in JTL Shop data to match Shopware IDs based on email,
    leaving all other columns completely unchanged.

    Args:
        shopware_file: Path to Shopware user data (CSV/Excel)
        jtl_file: Path to JTL Shop customer data (CSV/Excel)
        output_file: Path to save updated JTL data
    """
    try:
        print("Loading data files...")
        # Read files (autodetect CSV/Excel)
        shopware_df = pd.read_csv(shopware_file) if shopware_file.endswith('.csv') else pd.read_excel(shopware_file)
        jtl_df = pd.read_csv(jtl_file) if jtl_file.endswith('.csv') else pd.read_excel(jtl_file)

        print("Creating email to Shopware ID mapping...")
        # Create {email: shopware_id} dictionary
        email_to_shopware_id = {
            str(row['email']).lower(): row['id']
            for _, row in shopware_df.iterrows()
            if pd.notna(row['email'])
        }

        print("Processing JTL data...")
        changes = 0
        # Create copy to preserve original data
        updated_jtl_df = jtl_df.copy()

        for index, row in updated_jtl_df.iterrows():
            if pd.notna(row['cMail']):
                jtl_email = str(row['cMail']).lower()
                if jtl_email in email_to_shopware_id:
                    shopware_id = email_to_shopware_id[jtl_email]
                    if row['kKunde'] != shopware_id:
                        updated_jtl_df.at[index, 'kKunde'] = shopware_id
                        changes += 1
                        print(f"Updated {jtl_email}: {row['kKunde']} → {shopware_id}")

        # Save results
        if output_file.endswith('.csv'):
            updated_jtl_df.to_csv(output_file, index=False)
        else:
            updated_jtl_df.to_excel(output_file, index=False)

        print(f"\nProcess complete. {changes} IDs updated.")
        print(f"Original JTL records: {len(jtl_df)}")
        print(f"Updated file saved to: {output_file}")

    except Exception as e:
        print(f"\nError: {str(e)}")

# Configuration - update these paths
if __name__ == "__main__":
    SHOPWARE_DATA = "shopware_users.csv"  # or .xlsx
    JTL_DATA = "jtl_customers.csv"        # or .xlsx
    OUTPUT_FILE = "jtl_customers_updated.csv"  # or .xlsx

    print("Starting JTL kKunde ID update process...")
    update_jtl_ids_only(SHOPWARE_DATA, JTL_DATA, OUTPUT_FILE)

Streaming output truncated to the last 5000 lines.
Updated vschindlbeck@gmail.com: 4884 → 5173
Updated bernhardengel@gmx.de: 4885 → 5174
Updated ole.muehlfeld@gmx.de: 4886 → 5175
Updated julia@beischaefers.de: 4887 → 5176
Updated ursulafischer80@hotmail.com: 4888 → 5177
Updated ursula.prokopp@kps-partner.at: 4889 → 5178
Updated joho65@gmx.net: 4890 → 5179
Updated hermes.da@t-online.de: 4891 → 5180
Updated pep.haug@gmail.com: 4892 → 5182
Updated amazon@logo-cash.de: 4893 → 5183
Updated martin.wirt@gmx.de: 4894 → 5184
Updated idanzeige@gmail.com: 4895 → 5185
Updated schallhart@gmx.at: 4896 → 5186
Updated bastian.jann@gmx.de: 4897 → 5187
Updated d_schill@web.de: 4898 → 5188
Updated kfz-moritz@web.de: 4899 → 5189
Updated linda-87@t-online.de: 4900 → 5190
Updated flino@mail.de: 4901 → 5191
Updated roth_schuler@mac.com: 4902 → 5192
Updated tenidde2291@gmail.com: 4903 → 5193
Updated dr.petra.walter@gmx.de: 4904 → 5194
Updated eissler-eissler@web.de: 4905 → 5195
Updated maurinezaratin@hotmail.

In [ ]:
import pandas as pd
from datetime import datetime

def csv_to_sql_insert(csv_file_path, sql_file_path, table_name, batch_size=100):
    """
    Convert CSV data to SQL INSERT statements with strict NULL handling based on table structure.

    Args:
        csv_file_path: Path to the input CSV file
        sql_file_path: Path to save the SQL file
        table_name: Name of the target SQL table
        batch_size: Number of rows per INSERT statement
    """
    try:
        # Read CSV file
        df = pd.read_csv(csv_file_path)

        # Define column specifications based on your table structure
        column_specs = {
            'kKunde': {'null': False, 'default': None, 'type': 'int'},
            'kKundengruppe': {'null': False, 'default': '0', 'type': 'int'},
            'kSprache': {'null': False, 'default': '0', 'type': 'int'},
            'cKundenNr': {'null': True, 'default': 'NULL', 'type': 'str'},
            'cPasswort': {'null': True, 'default': 'NULL', 'type': 'str'},
            'cAnrede': {'null': False, 'default': "''", 'type': 'str'},
            'cTitel': {'null': True, 'default': 'NULL', 'type': 'str'},
            'cVorname': {'null': False, 'default': "''", 'type': 'str'},
            'cNachname': {'null': False, 'default': "''", 'type': 'str'},
            'cFirma': {'null': True, 'default': 'NULL', 'type': 'str'},
            'cZusatz': {'null': True, 'default': 'NULL', 'type': 'str'},
            'cStrasse': {'null': False, 'default': "''", 'type': 'str'},
            'cHausnummer': {'null': False, 'default': "''", 'type': 'str'},
            'cAdressZusatz': {'null': True, 'default': 'NULL', 'type': 'str'},
            'cPLZ': {'null': False, 'default': "''", 'type': 'str'},
            'cOrt': {'null': False, 'default': "''", 'type': 'str'},
            'cBundesland': {'null': False, 'default': "''", 'type': 'str'},
            'cLand': {'null': False, 'default': "''", 'type': 'str'},
            'cTel': {'null': True, 'default': 'NULL', 'type': 'str'},
            'cMobil': {'null': True, 'default': 'NULL', 'type': 'str'},
            'cFax': {'null': True, 'default': 'NULL', 'type': 'str'},
            'cMail': {'null': False, 'default': "''", 'type': 'str'},
            'cUSTID': {'null': True, 'default': 'NULL', 'type': 'str'},
            'cWWW': {'null': True, 'default': 'NULL', 'type': 'str'},
            'cSperre': {'null': False, 'default': "'N'", 'type': 'str'},
            'fGuthaben': {'null': False, 'default': '0', 'type': 'float'},
            'cNewsletter': {'null': False, 'default': "''", 'type': 'str'},
            'dGeburtstag': {'null': True, 'default': 'NULL', 'type': 'date'},
            'fRabatt': {'null': False, 'default': '0', 'type': 'float'},
            'cHerkunft': {'null': False, 'default': "''", 'type': 'str'},
            'dErstellt': {'null': True, 'default': 'NULL', 'type': 'date'},
            'dVeraendert': {'null': False, 'default': "CURRENT_TIMESTAMP", 'type': 'datetime'},
            'cAktiv': {'null': False, 'default': "'Y'", 'type': 'str'},
            'cAbgeholt': {'null': False, 'default': "'N'", 'type': 'str'},
            'nRegistriert': {'null': False, 'default': '0', 'type': 'int'},
            'nLoginversuche': {'null': False, 'default': '0', 'type': 'int'},
            'dLastLogin': {'null': True, 'default': 'NULL', 'type': 'datetime'},
            'dSessionInvalidate': {'null': True, 'default': 'NULL', 'type': 'datetime'},
            'b2FAauth': {'null': True, 'default': '0', 'type': 'int'},
            'c2FAauthSecret': {'null': True, 'default': "''", 'type': 'str'}
        }

        # Prepare SQL file
        with open(sql_file_path, 'w', encoding='utf-8') as sql_file:
            # Write SQL header
            columns = df.columns.tolist()
            sql_file.write(f"INSERT INTO `{table_name}` (\n")
            sql_file.write("    `" + "`, `".join(columns) + "`\n")
            sql_file.write(") VALUES\n")

            # Process rows in batches
            for i in range(0, len(df), batch_size):
                batch = df.iloc[i:i+batch_size]

                # Write each row in the batch
                for j, row in batch.iterrows():
                    values = []
                    for col in columns:
                        val = row[col]
                        spec = column_specs.get(col, {'null': True, 'default': 'NULL', 'type': 'str'})

                        # Handle NULL values based on column specs
                        if pd.isna(val):
                            if not spec['null']:
                                values.append(spec['default'])
                            else:
                                values.append('NULL')
                        else:
                            # Format value based on type
                            if spec['type'] == 'int':
                                values.append(str(int(val)))
                            elif spec['type'] == 'float':
                                values.append(str(float(val)))
                            elif spec['type'] == 'date':
                                if isinstance(val, str):
                                    try:
                                        dt = datetime.strptime(val, '%Y-%m-%d')
                                        values.append(f"'{dt.strftime('%Y-%m-%d')}'")
                                    except:
                                        values.append(spec['default'])
                                else:
                                    values.append(f"'{val.strftime('%Y-%m-%d')}'")
                            elif spec['type'] == 'datetime':
                                if isinstance(val, str):
                                    try:
                                        dt = datetime.strptime(val, '%Y-%m-%d %H:%M:%S')
                                        values.append(f"'{dt.strftime('%Y-%m-%d %H:%M:%S')}'")
                                    except:
                                        values.append(spec['default'])
                                else:
                                    values.append(f"'{val.strftime('%Y-%m-%d %H:%M:%S')}'")
                            else:  # string
                                val_str = str(val).replace("'", "''")
                                values.append(f"'{val_str}'")

                    # Write the VALUES line
                    sql_file.write("    (" + ", ".join(values) + ")")

                    # Add comma unless it's the last row in batch or file
                    if j != batch.index[-1] and j != df.index[-1]:
                        sql_file.write(",\n")
                    else:
                        sql_file.write(";\n\n")

                        # If more batches coming, start new INSERT statement
                        if i + batch_size < len(df):
                            sql_file.write(f"INSERT INTO `{table_name}` (\n")
                            sql_file.write("    `" + "`, `".join(columns) + "`\n")
                            sql_file.write(") VALUES\n")

        print(f"Successfully converted CSV to SQL INSERT statements. Saved to: {sql_file_path}")
        print(f"Total rows processed: {len(df)}")

    except Exception as e:
        print(f"Error: {str(e)}")

# Configuration
if __name__ == "__main__":
    CSV_FILE = "jtl_customers_updated.csv"  # Your input CSV file
    SQL_FILE = "jtl_customers_inserts.sql"  # Output SQL file
    TABLE_NAME = "tkunde"  # Your target table name
    BATCH_SIZE = 100  # Rows per INSERT statement

    print("Starting CSV to SQL conversion with strict NULL handling...")
    csv_to_sql_insert(CSV_FILE, SQL_FILE, TABLE_NAME, BATCH_SIZE)

Starting CSV to SQL conversion with strict NULL handling...
Successfully converted CSV to SQL INSERT statements. Saved to: jtl_customers_inserts.sql
Total rows processed: 9879


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import chardet
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filename='shopware_to_jtl.log',
    filemode='w'
)
logger = logging.getLogger()

def detect_encoding(file_path):
    """Detect file encoding using chardet"""
    try:
        with open(file_path, 'rb') as f:
            result = chardet.detect(f.read())
        return result['encoding']
    except Exception as e:
        logger.error(f"Error detecting encoding for {file_path}: {str(e)}")
        return 'utf-8'  # Fallback to utf-8

def convert_shopware_to_jtl_orders(shopware_csv, jtl_customers_csv, output_prefix):
    """
    Complete conversion of Shopware 5 orders to JTL Shop 5 format.
    Matches customers by customerId (Shopware) to kKunde (JTL).
    """
    try:
        logger.info("Starting conversion process...")

        # Detect and load files with proper encoding
        shopware_encoding = detect_encoding(shopware_csv)
        jtl_encoding = detect_encoding(jtl_customers_csv)

        logger.info(f"Loading {shopware_csv} with {shopware_encoding} encoding")
        shopware_orders = pd.read_csv(shopware_csv, decimal=',', encoding=shopware_encoding)

        logger.info(f"Loading {jtl_customers_csv} with {jtl_encoding} encoding")
        jtl_customers = pd.read_csv(jtl_customers_csv, encoding=jtl_encoding)

        # Create mapping from customerId to kKunde (they should be the same)
        customer_id_to_kKunde = dict(zip(
            jtl_customers['kKunde'],
            jtl_customers['kKunde']
        ))

        # Initialize DataFrames for JTL tables
        columns = {
            'tlieferadresse': [
                'kLieferadresse', 'kKunde', 'cAnrede', 'cVorname', 'cNachname',
                'cTitel', 'cFirma', 'cZusatz', 'cStrasse', 'cHausnummer',
                'cAdressZusatz', 'cPLZ', 'cOrt', 'cBundesland', 'cLand',
                'cTel', 'cMobil', 'cFax', 'cMail'
            ],
            'twarenkorb': [
                'kWarenkorb', 'kKunde', 'kLieferadresse', 'kZahlungsInfo'
            ],
            'twarenkorbpos': [
                'kWarenkorbPos', 'kWarenkorb', 'kArtikel', 'kVersandklasse',
                'cName', 'cLieferstatus', 'cArtNr', 'cEinheit', 'fPreisEinzelNetto',
                'fPreis', 'fMwSt', 'nAnzahl', 'nPosTyp', 'cHinweis', 'cUnique',
                'cResponsibility', 'kKonfigitem', 'kBestellpos', 'fLagerbestandVorAbschluss',
                'nLongestMinDelivery', 'nLongestMaxDelivery'
            ],
            'twarenkorbpers': [
                'kKunde', 'dErstellt'
            ]
        }

        tables = {
            'tlieferadresse': pd.DataFrame(columns=columns['tlieferadresse']),
            'twarenkorb': pd.DataFrame(columns=columns['twarenkorb']),
            'twarenkorbpos': pd.DataFrame(columns=columns['twarenkorbpos']),
            'twarenkorbpers': pd.DataFrame(columns=columns['twarenkorbpers'])
        }

        # Initialize IDs
        current_ids = {
            'lieferadresse': 3,
            'warenkorb': 5,
            'warenkorbpos': 26
        }

        # Statistics
        stats = {
            'total_orders': 0,
            'matched_orders': 0,
            'skipped_orders': 0,
            'total_items': 0
        }

        logger.info("Processing orders...")

        # Process each unique order (group by orderId)
        for order_id, order_group in shopware_orders.groupby('orderId'):
            stats['total_orders'] += 1
            first_row = order_group.iloc[0]
            customer_id = first_row['customerId']

            # Match customer by customerId (Shopware) to kKunde (JTL)
            kKunde = customer_id_to_kKunde.get(customer_id)

            if kKunde is None:
                logger.warning(f"No JTL customer found for order {order_id} with customerId {customer_id}")
                stats['skipped_orders'] += 1
                continue

            stats['matched_orders'] += 1

            # Parse order time (handle multiple date formats)
            order_time = None
            for fmt in ('%d/%m/%Y %H:%M', '%m/%d/%Y %H:%M', '%Y-%m-%d %H:%M:%S'):
                try:
                    order_time = datetime.strptime(first_row['orderTime'], fmt)
                    break
                except ValueError:
                    continue

            if order_time is None:
                logger.warning(f"Could not parse date for order {order_id}: {first_row['orderTime']}")
                order_time = datetime.now()

            # 1. tlieferadresse (shipping address)
            tables['tlieferadresse'].loc[len(tables['tlieferadresse'])] = {
                'kLieferadresse': current_ids['lieferadresse'],
                'kKunde': kKunde,
                'cAnrede': first_row.get('salutation', 'm'),
                'cVorname': first_row.get('firstname', ''),
                'cNachname': first_row.get('lastname', ''),
                'cTitel': first_row.get('title', ''),
                'cFirma': first_row.get('company', ''),
                'cZusatz': first_row.get('additional_address_line1', ''),
                'cStrasse': first_row.get('street', ''),
                'cHausnummer': first_row.get('streetnumber', '8'),
                'cAdressZusatz': first_row.get('additional_address_line2', ''),
                'cPLZ': first_row.get('zipcode', '48527'),
                'cOrt': first_row.get('city', 'Nordhorn'),
                'cBundesland': first_row.get('state', ''),
                'cLand': first_row.get('country', 'DE'),
                'cTel': first_row.get('phone', ''),
                'cMobil': first_row.get('mobile', ''),
                'cFax': first_row.get('fax', ''),
                'cMail': first_row.get('email', '')
            }

            # 2. twarenkorb (order header)
            tables['twarenkorb'].loc[len(tables['twarenkorb'])] = {
                'kWarenkorb': current_ids['warenkorb'],
                'kKunde': kKunde,
                'kLieferadresse': current_ids['lieferadresse'],
                'kZahlungsInfo': first_row['paymentID']  # Use Shopware payment ID directly
            }

            # 3. twarenkorbpos (order items)
            for _, item in order_group.iterrows():
                stats['total_items'] += 1

                # Convert price from string to float (handling EU decimal format)
                try:
                    price = float(str(item['price']).replace(',', '.'))
                except:
                    price = 0.0
                    logger.warning(f"Could not parse price for item {item['articleNumber']} in order {order_id}")

                # Main product (PosTyp 1)
                tables['twarenkorbpos'].loc[len(tables['twarenkorbpos'])] = {
                    'kWarenkorbPos': current_ids['warenkorbpos'],
                    'kWarenkorb': current_ids['warenkorb'],
                    'kArtikel': 0,  # Needs mapping to JTL articles
                    'kVersandklasse': 6 if current_ids['warenkorbpos'] % 2 == 0 else 1,
                    'cName': str(item['articleName']),
                    'cLieferstatus': 'completed',
                    'cArtNr': str(item['articleNumber']),
                    'cEinheit': 'Stück',
                    'fPreisEinzelNetto': price,
                    'fPreis': price * int(item['quantity']),
                    'fMwSt': float(item['taxRate']),
                    'nAnzahl': int(item['quantity']),
                    'nPosTyp': 1,  # 1=article
                    'cHinweis': '',
                    'cUnique': 'core',
                    'cResponsibility': '',
                    'kKonfigitem': 0,
                    'kBestellpos': 87000 + current_ids['warenkorbpos'],
                    'fLagerbestandVorAbschluss': np.nan,
                    'nLongestMinDelivery': 2,
                    'nLongestMaxDelivery': 3
                }
                current_ids['warenkorbpos'] += 1

            # Add shipping cost (PosTyp 2)
            try:
                shipping_net = float(str(first_row['invoiceShippingNet']).replace(',', '.'))
                shipping = float(str(first_row['invoiceShipping']).replace(',', '.'))
            except:
                shipping_net = 4.19
                shipping = 4.99
                logger.warning(f"Could not parse shipping costs for order {order_id}")

            tables['twarenkorbpos'].loc[len(tables['twarenkorbpos'])] = {
                'kWarenkorbPos': current_ids['warenkorbpos'],
                'kWarenkorb': current_ids['warenkorb'],
                'kArtikel': 0,
                'kVersandklasse': 0,
                'cName': 'DHL',  # Default shipping method
                'cLieferstatus': 'completed',
                'cArtNr': '',
                'cEinheit': 'Stück',
                'fPreisEinzelNetto': shipping_net,
                'fPreis': shipping,
                'fMwSt': shipping - shipping_net,
                'nAnzahl': 1,
                'nPosTyp': 2,  # 2=shipping
                'cHinweis': '',
                'cUnique': 'core',
                'cResponsibility': '',
                'kKonfigitem': 0,
                'kBestellpos': 87000 + current_ids['warenkorbpos'],
                'fLagerbestandVorAbschluss': np.nan,
                'nLongestMinDelivery': 0,
                'nLongestMaxDelivery': 0
            }
            current_ids['warenkorbpos'] += 1

            # 4. twarenkorbpers (order person)
            tables['twarenkorbpers'].loc[len(tables['twarenkorbpers'])] = {
                'kKunde': kKunde,
                'dErstellt': order_time.strftime('%Y-%m-%d %H:%M:%S')
            }

            # Increment IDs for next order
            current_ids['lieferadresse'] += 1
            current_ids['warenkorb'] += 1

        logger.info("Saving output files...")
        # Save all tables to CSV with UTF-8 encoding
        for table_name, df in tables.items():
            output_file = f"{output_prefix}_{table_name}.csv"
            df.to_csv(output_file, index=False, encoding='utf-8')
            logger.info(f"Saved {output_file} with {len(df)} records")

        # Print summary statistics
        logger.info("\nConversion Summary:")
        logger.info(f"Total Orders Processed: {stats['total_orders']}")
        logger.info(f"Successfully Matched Orders: {stats['matched_orders']}")
        logger.info(f"Skipped Orders (no customer match): {stats['skipped_orders']}")
        logger.info(f"Total Order Items Processed: {stats['total_items']}")
        logger.info(f"Generated files with prefix: {output_prefix}_*.csv")

        print("\nConversion completed successfully! Check shopware_to_jtl.log for details.")

    except Exception as e:
        logger.error(f"Fatal error during conversion: {str(e)}", exc_info=True)
        print(f"\nError: {str(e)}\nCheck shopware_to_jtl.log for details.")

if __name__ == "__main__":
    # Configuration - update these paths
    SHOPWARE_CSV = "shopware_order_07.csv"       # Path to Shopware orders CSV
    JTL_CUSTOMERS_CSV = "jtl_customers.csv"    # Path to JTL customers CSV
    OUTPUT_PREFIX = "jtl_orders"               # Prefix for output files

    print("=== Shopware 5 to JTL Shop 5 Order Conversion ===")
    print("This script will convert Shopware orders to JTL format.")
    print(f"Input files: {SHOPWARE_CSV}, {JTL_CUSTOMERS_CSV}")
    print(f"Output prefix: {OUTPUT_PREFIX}_*.csv")
    print("Detailed logs will be written to shopware_to_jtl.log\n")

    convert_shopware_to_jtl_orders(SHOPWARE_CSV, JTL_CUSTOMERS_CSV, OUTPUT_PREFIX)

=== Shopware 5 to JTL Shop 5 Order Conversion ===
This script will convert Shopware orders to JTL format.
Input files: shopware_order_07.csv, jtl_customers.csv
Output prefix: jtl_orders_*.csv
Detailed logs will be written to shopware_to_jtl.log




Conversion completed successfully! Check shopware_to_jtl.log for details.


In [ ]:
import pandas as pd

# Load the CSV files
tlieferadresse_df = pd.read_csv('tlieferadresse.csv')
customer_df = pd.read_csv('jtl_customers_updated.csv')

# Merge the dataframes on kKunde
merged_df = pd.merge(
    tlieferadresse_df,
    customer_df[['kKunde', 'cAnrede', 'cVorname', 'cNachname', 'cTitel', 'cFirma',
                 'cZusatz', 'cStrasse', 'cHausnummer', 'cAdressZusatz', 'cPLZ',
                 'cOrt', 'cBundesland', 'cLand', 'cTel', 'cMobil', 'cFax', 'cMail']],
    on='kKunde',
    how='left',
    suffixes=('', '_customer')
)

# Update columns where customer data exists
for column in ['cAnrede', 'cVorname', 'cNachname', 'cTitel', 'cFirma', 'cZusatz',
               'cStrasse', 'cHausnummer', 'cAdressZusatz', 'cPLZ', 'cOrt',
               'cBundesland', 'cLand', 'cTel', 'cMobil', 'cFax', 'cMail']:
    merged_df[column] = merged_df[column+'_customer'].combine_first(merged_df[column])
    merged_df.drop(column+'_customer', axis=1, inplace=True)

# Save the updated dataframe
merged_df.to_csv('updated_tlieferadresse.csv', index=False)

print("Mapping completed. Updated file saved as 'updated_tlieferadresse.csv'")

Mapping completed. Updated file saved as 'updated_tlieferadresse.csv'


In [ ]:
import csv

def csv_to_tlieferadresse_sql(csv_file, output_file):
    # Define all columns in tlieferadresse table with their null constraints
    # Format: (column_name, default_value_for_null)
    tlieferadresse_columns = [
        ('kLieferadresse', None),  # Primary key, must have value
        ('kKunde', None),          # Required field
        ('cAnrede', "''"),         # varchar(20), No
        ('cVorname', "''"),        # varchar(255), No
        ('cNachname', "''"),       # varchar(255), No
        ('cTitel', 'NULL'),        # varchar(64), Yes
        ('cFirma', 'NULL'),        # varchar(255), Yes
        ('cZusatz', 'NULL'),       # varchar(255), Yes
        ('cStrasse', "''"),        # varchar(255), No
        ('cHausnummer', "''"),     # varchar(32), No
        ('cAdressZusatz', 'NULL'), # varchar(255), Yes
        ('cPLZ', "''"),            # varchar(20), No
        ('cOrt', "''"),            # varchar(255), No
        ('cBundesland', "''"),     # varchar(255), No - This was the problem
        ('cLand', "''"),           # varchar(255), No
        ('cTel', 'NULL'),          # varchar(255), Yes
        ('cMobil', 'NULL'),        # varchar(255), Yes
        ('cFax', 'NULL'),          # varchar(255), Yes
        ('cMail', 'NULL')          # varchar(255), Yes
    ]

    with open(csv_file, 'r', encoding='utf-8') as csvfile, \
         open(output_file, 'w', encoding='utf-8') as sqlfile:

        reader = csv.DictReader(csvfile)

        for row in reader:
            values = []
            for column, default_null in tlieferadresse_columns:
                if column not in row or not row[column] or row[column].lower() == 'null':
                    if default_null == 'NULL':
                        values.append('NULL')
                    else:
                        values.append(default_null)
                else:
                    escaped_value = row[column].replace("'", "''")
                    values.append(f"'{escaped_value}'")

            # Create the column list with backticks
            columns = [f"`{col[0]}`" for col in tlieferadresse_columns]

            sql = f"INSERT INTO `tlieferadresse` ({', '.join(columns)}) \n"
            sql += f"VALUES ({', '.join(values)});\n"
            sqlfile.write(sql + '\n')

# Usage example:
csv_to_tlieferadresse_sql('updated_tlieferadresse.csv', 'tlieferadresse_inserts.sql')

In [ ]:
import csv

def csv_to_sql_insert(csv_file, output_file, table_name='twarenkorb'):
    with open(csv_file, 'r', encoding='utf-8') as csvfile, \
         open(output_file, 'w', encoding='utf-8') as sqlfile:

        reader = csv.DictReader(csvfile)

        for row in reader:
            # Prepare column names and values
            columns = ['kWarenkorb', 'kKunde', 'kLieferadresse', 'kZahlungsInfo']

            # Handle NULL values for kZahlungsInfo
            zahlungsinfo = row.get('kZahlungsInfo', '0')
            if zahlungsinfo.lower() == 'null' or zahlungsinfo.strip() == '':
                zahlungsinfo = 'NULL'

            values = [
                row.get('kWarenkorb', '0'),  # Will be auto-incremented if not provided
                row.get('kKunde', '0'),
                row.get('kLieferadresse', '0'),
                zahlungsinfo
            ]

            # Build the SQL query
            sql = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ("
            sql += ', '.join(values if values[3] == 'NULL' else [f"'{v}'" for v in values])
            sql += ");\n"

            sqlfile.write(sql)

# Example usage:
csv_to_sql_insert('warenkorb_data.csv', 'warenkorb_inserts.sql')

In [ ]:
import csv

def generate_corrected_warenkorbpos_sql(tartikel_csv, twarenkorbpos_csv, output_sql):
    # Step 1: Build a mapping dictionary from cArtNr to kArtikel from tartikel
    artnr_to_kartikel = {}

    with open(tartikel_csv, 'r', encoding='utf-8') as artikel_file:
        artikel_reader = csv.DictReader(artikel_file, delimiter='\t')
        for row in artikel_reader:
            artnr = row.get('cArtNr', '').strip()
            kartikel = row.get('kArtikel', '').strip()
            if artnr and kartikel:
                artnr_to_kartikel[artnr] = kartikel

    # Step 2: Process twarenkorbpos and generate corrected SQL
    with open(twarenkorbpos_csv, 'r', encoding='utf-8') as warenkorbpos_file, \
         open(output_sql, 'w', encoding='utf-8') as sql_file:

        warenkorbpos_reader = csv.DictReader(warenkorbpos_file, delimiter='\t')

        for row in warenkorbpos_reader:
            # Get the original values
            original_values = row.copy()
            cArtNr = original_values.get('cArtNr', '').strip()

            # Update only the kArtikel field if we find a match
            if cArtNr in artnr_to_kartikel:
                original_values['kArtikel'] = artnr_to_kartikel[cArtNr]

            # Prepare the SQL values
            columns = []
            values = []

            for col, val in original_values.items():
                columns.append(col)
                if val == '' or val.lower() == 'null':
                    values.append('NULL')
                else:
                    values.append(f"'{val}'")

            # Generate the SQL INSERT statement
            sql = f"INSERT INTO twarenkorbpos ({', '.join(columns)}) VALUES ({', '.join(values)});\n"
            sql_file.write(sql)

# Example usage:
generate_corrected_warenkorbpos_sql(
    tartikel_csv='tartikel.csv',
    twarenkorbpos_csv='twarenkorbpos.csv',
    output_sql='corrected_warenkorbpos_inserts.sql'
)

In [42]:
import pandas as pd
import re

def clean_sql_value(val):
    if pd.isna(val) or val == '':
        return 'NULL'
    if isinstance(val, (int, float)):
        return str(val)
    val_str = str(val)
    # Escape special characters for SQL
    val_str = val_str.replace("\\", "\\\\").replace("'", "''")
    # Remove any remaining non-printable characters
    val_str = re.sub(r'[\x00-\x1F\x7F-\x9F]', '', val_str)
    return f"'{val_str}'"

def generate_insert_statements(csv_path, output_sql_path, batch_size=100):
    try:
        # Read CSV with encoding fallback
        encodings = ['utf-8', 'latin1', 'iso-8859-1', 'cp1252']
        df = None
        for encoding in encodings:
            try:
                df = pd.read_csv(csv_path, encoding=encoding, keep_default_na=False)
                break
            except UnicodeDecodeError:
                continue

        if df is None:
            raise ValueError("Could not read CSV file with any of the tried encodings")

        # Define default values for NOT NULL columns
        column_defaults = {
            'kWarenkorb': '0',
            'kArtikel': '0',
            'kVersandklasse': '1',
            'cLieferstatus': "''",
            'cArtNr': "''",
            'cEinheit': "''",
            'fPreisEinzelNetto': '0',
            'fPreis': '0',
            'nAnzahl': '0',
            'nPosTyp': '1',
            'cHinweis': "''",
            'cUnique': "''",
            'cResponsibility': "'core'",
            'kKonfigitem': '0',
            'kBestellpos': '0',
            'nLongestMinDelivery': '0',
            'nLongestMaxDelivery': '0'
        }

        with open(output_sql_path, 'w', encoding='utf-8') as sql_file:
            # Write header
            sql_file.write("-- AUTO-GENERATED INSERT STATEMENTS FOR `twarenkorbpos`\n")
            sql_file.write("-- Generated from: " + csv_path + "\n\n")
            sql_file.write("SET NAMES utf8mb4;\n")
            sql_file.write("SET FOREIGN_KEY_CHECKS = 0;\n\n")

            for i in range(0, len(df), batch_size):
                batch = df.iloc[i:i+batch_size]

                # Determine columns to include (exclude auto-increment if present)
                columns_to_include = [col for col in batch.columns if col != 'kWarenkorbPos']
                columns = ', '.join([f'`{col}`' for col in columns_to_include])

                value_rows = []
                for _, row in batch.iterrows():
                    values = []
                    for col in columns_to_include:
                        val = row[col]
                        if pd.isna(val) or val == '':
                            values.append(column_defaults.get(col, 'NULL'))
                        else:
                            values.append(clean_sql_value(val))
                    value_rows.append(f"({', '.join(values)})")

                insert_stmt = (
                    f"INSERT INTO `twarenkorbpos` ({columns})\n"
                    f"VALUES {', '.join(value_rows)};\n"
                )
                sql_file.write(insert_stmt + "\n")

            sql_file.write("\nSET FOREIGN_KEY_CHECKS = 1;\n")

        print(f"Success! MySQL INSERT statements generated at {output_sql_path}")
        return True

    except Exception as e:
        print(f"Error generating SQL: {str(e)}")
        return False

# Example usage
generate_insert_statements(
    csv_path='updated_output.csv',
    output_sql_path='twarenkorbpos_inserts.sql'
)

Success! MySQL INSERT statements generated at twarenkorbpos_inserts.sql


True

In [ ]:
import csv
from collections import defaultdict
import os
import sys
import pandas as pd

def read_csv_with_fallback(file_path, expected_headers=None):
    """Attempt to read CSV with multiple encoding and delimiter options"""
    encodings = ['utf-8-sig', 'utf-8', 'latin1', 'iso-8859-1', 'windows-1252']
    delimiters = ['\t', ',', ';']

    for encoding in encodings:
        for delimiter in delimiters:
            try:
                with open(file_path, 'r', encoding=encoding) as f:
                    # Peek at first line to verify headers
                    first_line = f.readline()
                    f.seek(0)

                    reader = csv.DictReader(f, delimiter=delimiter)
                    if not reader.fieldnames:
                        continue

                    if expected_headers:
                        missing_headers = set(expected_headers) - set(reader.fieldnames)
                        if missing_headers:
                            continue

                    print(f"Successfully opened {file_path} with encoding={encoding}, delimiter={repr(delimiter)}")
                    return [row for row in reader], reader.fieldnames

            except Exception as e:
                continue

    raise ValueError(f"Could not read {file_path} with any combination of encodings and delimiters")

def build_multi_column_mapping(tartikel_path):
    """Build mapping dictionary using all possible identifier columns"""
    mapping_sources = {
        'cArtNr': 'kArtikel',
        'cEAN': 'kArtikel',
        'cName': 'kArtikel',
        'cHAN': 'kArtikel'
    }

    mappings = {col: {} for col in mapping_sources.keys()}
    tartikel_data, _ = read_csv_with_fallback(tartikel_path)

    for row in tartikel_data:
        for src_col, target_col in mapping_sources.items():
            if src_col in row and target_col in row:
                key = row[src_col].strip()
                if key:
                    mappings[src_col][key] = row[target_col].strip()

    return mappings

def map_warenkorb_with_fallback(warenkorbpos_path, mappings, output_path):
    """Try multiple columns to find matches"""
    stats = {
        'total_rows': 0,
        'skipped': 0,
        'matched': defaultdict(int),
        'unmatched': 0
    }

    warenkorb_data, headers = read_csv_with_fallback(warenkorbpos_path)

    with open(output_path, 'w', encoding='utf-8', newline='') as out_file:
        writer = csv.DictWriter(out_file, fieldnames=headers, delimiter='\t')
        writer.writeheader()

        for row in warenkorb_data:
            stats['total_rows'] += 1
            matched = False

            # Try each possible mapping column
            for map_col in mappings.keys():
                if map_col in row:
                    key = row[map_col].strip()
                    if key and key in mappings[map_col]:
                        row['kArtikel'] = mappings[map_col][key]
                        stats['matched'][map_col] += 1
                        matched = True
                        break

            if not matched:
                stats['unmatched'] += 1
                if all(col not in row for col in mappings.keys()):
                    stats['skipped'] += 1

            writer.writerow(row)

    return stats

def generate_diagnostic_report(input_path):
    """Generate detailed report about CSV structure"""
    report = []

    try:
        df = pd.read_csv(input_path, delimiter='\t', encoding='latin1', nrows=1000)
        report.append(f"\nDiagnostic Report for {input_path}")
        report.append(f"Total rows: {len(df)}")
        report.append(f"Columns: {list(df.columns)}")

        report.append("\nColumn Analysis:")
        for col in df.columns:
            non_empty = df[col].notna().sum()
            unique = df[col].nunique()
            sample = df[col].dropna().sample(min(5, len(df))) if non_empty > 0 else []
            report.append(
                f"{col}: {non_empty} non-empty, {unique} unique values\n"
                f"Sample: {list(sample)}"
            )

    except Exception as e:
        report.append(f"Error generating diagnostic: {str(e)}")

    return "\n".join(report)

def main():
    # Configuration
    input_files = {
        'tartikel_path': 'tartikel.csv',
        'warenkorbpos_path': 'twarenkorbpos.csv'
    }
    output_files = {
        'output_path': 'mapped_warenkorbpos_final.csv',
        'sql_path': 'warenkorbpos_inserts_final.sql',
        'report_path': 'diagnostic_report.txt'
    }

    # Verify files
    for path in input_files.values():
        if not os.path.exists(path):
            print(f"Error: Input file {path} not found!")
            sys.exit(1)

    # Generate diagnostic report
    with open(output_files['report_path'], 'w') as report_file:
        report_file.write(generate_diagnostic_report(input_files['warenkorbpos_path']))
        report_file.write("\n\n")
        report_file.write(generate_diagnostic_report(input_files['tartikel_path']))

    print("Starting mapping process...")

    # Build comprehensive mappings
    mappings = build_multi_column_mapping(input_files['tartikel_path'])

    # Process with fallback matching
    stats = map_warenkorb_with_fallback(
        input_files['warenkorbpos_path'],
        mappings,
        output_files['output_path']
    )

    # Print results
    print("\n=== Mapping Results ===")
    print(f"Total rows processed: {stats['total_rows']}")
    print(f"Skipped (no matching columns): {stats['skipped']}")
    print(f"Matched: {sum(stats['matched'].values())}")
    for col, count in stats['matched'].items():
        print(f"  - By {col}: {count}")
    print(f"Unmatched (columns found but no match): {stats['unmatched']}")

    # Generate SQL
    print("\nGenerating SQL...")
    generate_sql(
        csv_path=output_files['output_path'],
        sql_path=output_files['sql_path']
    )

    print("\nProcessing complete!")
    print(f"Diagnostic report saved to {output_files['report_path']}")

def generate_sql(csv_path, sql_path, table_name='twarenkorbpos'):
    """Generate SQL INSERT statements"""
    try:
        with open(csv_path, 'r', encoding='utf-8') as csv_file:
            delimiter = '\t' if '\t' in csv_file.readline() else ','
            csv_file.seek(0)
            reader = csv.DictReader(csv_file, delimiter=delimiter)

            with open(sql_path, 'w', encoding='utf-8') as sql_file:
                for row in reader:
                    columns = []
                    values = []

                    for col, val in row.items():
                        columns.append(col)
                        if not val or val.lower() == 'null':
                            values.append('NULL')
                        else:
                            val = str(val).replace("'", "''")
                            values.append(f"'{val}'")

                    sql = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({', '.join(values)});\n"
                    sql_file.write(sql)

        print(f"SQL file generated at {sql_path}")
    except Exception as e:
        print(f"Error generating SQL: {str(e)}")
        sys.exit(1)

if __name__ == "__main__":
    main()

Starting mapping process...
Successfully opened tartikel.csv with encoding=utf-8-sig, delimiter='\t'
Successfully opened twarenkorbpos.csv with encoding=latin1, delimiter='\t'

=== Mapping Results ===
Total rows processed: 51744
Skipped (no matching columns): 51744
Matched: 0
Unmatched (columns found but no match): 51744

Generating SQL...
SQL file generated at warenkorbpos_inserts_final.sql

Processing complete!
Diagnostic report saved to diagnostic_report.txt


In [1]:
import pandas as pd
from datetime import datetime
import re

def clean_sql_value(val, col_type='string'):
    """Clean and format values for SQL insertion with type-specific handling"""
    if pd.isna(val) or val == '':
        return 'NULL'

    try:
        if col_type == 'datetime':
            dt = pd.to_datetime(val)
            return f"'{dt.strftime('%Y-%m-%d %H:%M:%S')}'"
        elif col_type == 'int':
            return str(int(float(val)))
        elif col_type == 'float':
            return str(float(val))
        else:  # string
            val_str = str(val)
            val_str = val_str.replace("\\", "\\\\").replace("'", "''")
            val_str = re.sub(r'[\x00-\x1F\x7F-\x9F]', '', val_str)  # Remove non-printables
            return f"'{val_str}'"
    except:
        return 'NULL'

def generate_twarenkorbpers_inserts(csv_path, output_sql_path, batch_size=100):
    try:
        # Read CSV with encoding fallback and explicit dtype handling
        encodings = ['utf-8', 'latin1', 'iso-8859-1', 'cp1252']
        df = None
        for encoding in encodings:
            try:
                df = pd.read_csv(
                    csv_path,
                    encoding=encoding,
                    keep_default_na=False,
                    dtype={'kKunde': 'Int64', 'kWarenkorbPers': 'Int64'},
                    parse_dates=['dErstellt'],
                    dayfirst=True  # Important for European date formats
                )
                break
            except UnicodeDecodeError:
                continue

        if df is None:
            raise ValueError("Could not read CSV file with any of the tried encodings")

        # Column type mapping for proper SQL formatting
        column_types = {
            'kWarenkorbPers': 'int',
            'kKunde': 'int',
            'dErstellt': 'datetime'
        }

        # Add kWarenkorbPers column starting from 2 if it doesn't exist
        if 'kWarenkorbPers' not in df.columns:
            df['kWarenkorbPers'] = range(2, len(df) + 2)
        else:
            # Ensure existing values start from 2
            min_val = df['kWarenkorbPers'].min()
            if pd.isna(min_val) or min_val < 2:
                df['kWarenkorbPers'] = range(2, len(df) + 2)

        with open(output_sql_path, 'w', encoding='utf-8') as sql_file:
            # Write SQL header with MariaDB/MySQL compatibility settings
            sql_file.write("/*!40101 SET @OLD_CHARACTER_SET_CLIENT=@@CHARACTER_SET_CLIENT */;\n")
            sql_file.write("/*!40101 SET @OLD_CHARACTER_SET_RESULTS=@@CHARACTER_SET_RESULTS */;\n")
            sql_file.write("/*!40101 SET @OLD_COLLATION_CONNECTION=@@COLLATION_CONNECTION */;\n")
            sql_file.write("/*!40101 SET NAMES utf8mb4 */;\n")
            sql_file.write("/*!40014 SET @OLD_FOREIGN_KEY_CHECKS=@@FOREIGN_KEY_CHECKS, FOREIGN_KEY_CHECKS=0 */;\n")
            sql_file.write("/*!40101 SET @OLD_SQL_MODE=@@SQL_MODE, SQL_MODE='NO_AUTO_VALUE_ON_ZERO' */;\n\n")

            sql_file.write(f"-- AUTO-GENERATED INSERT STATEMENTS FOR `twarenkorbpers`\n")
            sql_file.write(f"-- Generated at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            sql_file.write(f"-- Source: {csv_path}\n\n")

            # Disable auto-increment temporarily to insert specific values
            sql_file.write("SET @OLD_AUTOCOMMIT=@@AUTOCOMMIT, AUTOCOMMIT=0;\n")
            sql_file.write("SET @OLD_UNIQUE_CHECKS=@@UNIQUE_CHECKS, UNIQUE_CHECKS=0;\n\n")

            for i in range(0, len(df), batch_size):
                batch = df.iloc[i:i+batch_size]

                # Include all columns including kWarenkorbPers
                columns = ', '.join([f'`{col}`' for col in batch.columns])

                value_rows = []
                for idx, (_, row) in enumerate(batch.iterrows(), start=2+i):
                    values = []
                    for col in batch.columns:
                        val = row[col]
                        col_type = column_types.get(col, 'string')

                        # Special handling for kWarenkorbPers to ensure it starts from 2
                        if col == 'kWarenkorbPers':
                            values.append(str(idx))
                        else:
                            values.append(clean_sql_value(val, col_type))

                    value_rows.append(f"({', '.join(values)})")

                # Generate INSERT with explicit values
                insert_stmt = (
                    f"INSERT INTO `twarenkorbpers` ({columns}) VALUES\n"
                    + ",\n".join(value_rows) + ";\n\n"
                )
                sql_file.write(insert_stmt)

            # Restore settings
            sql_file.write("\nSET UNIQUE_CHECKS=@OLD_UNIQUE_CHECKS;\n")
            sql_file.write("SET AUTOCOMMIT=@OLD_AUTOCOMMIT;\n")
            sql_file.write("/*!40101 SET SQL_MODE=@OLD_SQL_MODE */;\n")
            sql_file.write("/*!40014 SET FOREIGN_KEY_CHECKS=@OLD_FOREIGN_KEY_CHECKS */;\n")
            sql_file.write("/*!40101 SET CHARACTER_SET_CLIENT=@OLD_CHARACTER_SET_CLIENT */;\n")
            sql_file.write("/*!40101 SET CHARACTER_SET_RESULTS=@OLD_CHARACTER_SET_RESULTS */;\n")
            sql_file.write("/*!40101 SET COLLATION_CONNECTION=@OLD_COLLATION_CONNECTION */;\n")

        print(f"Success! MySQL/MariaDB INSERT statements generated at {output_sql_path}")
        return True

    except Exception as e:
        print(f"Error generating SQL: {str(e)}")
        return False

# Example usage
generate_twarenkorbpers_inserts(
    csv_path='jtl_orders_twarenkorbpers.csv',
    output_sql_path='twarenkorbpers_inserts.sql'
)

Success! MySQL/MariaDB INSERT statements generated at twarenkorbpers_inserts.sql


True